In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# For efficiency with large files, consider reading in chunks:
chunk_iter = pd.read_csv('311.csv', chunksize=10000000)

# Read the first chunk
df_chunk = next(chunk_iter)

# Print schema (dtypes) and first 5 rows
print(df_chunk.dtypes)
print(df_chunk.head(5))


C:\Users\User\AppData\Local\Temp\ipykernel_15884\787093796.py:16: DtypeWarning: Columns (8,18,20,31,32,33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chunk = next(chunk_iter)


Unique Key                          int64
Created Date                       object
Closed Date                        object
Agency                             object
Agency Name                        object
Complaint Type                     object
Descriptor                         object
Location Type                      object
Incident Zip                       object
Incident Address                   object
Street Name                        object
Cross Street 1                     object
Cross Street 2                     object
Intersection Street 1              object
Intersection Street 2              object
Address Type                       object
City                               object
Landmark                           object
Facility Type                      object
Status                             object
Due Date                           object
Resolution Description             object
Resolution Action Updated Date     object
Community Board                   

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


# Define categorical columns
categorical_cols = [
    "Agency",
    "Complaint Type",
    "Descriptor",
    "Location Type",
    "City",
    "Borough",
    "Open Data Channel Type",
    "Community Board",
    "Facility Type",
]

# Process each chunk
processed_chunks = []

for chunk in chunk_iter:
    # Delete rows where 'Closed Date' is null
    chunk = chunk.dropna(subset=['Closed Date', 'Latitude', 'Longitude'])

    # Replace null values or "N/A" with "Unknown" for categorical columns
    for col_name in categorical_cols:
        chunk[col_name] = chunk[col_name].fillna("Unknown")
        chunk.loc[chunk[col_name] == "N/A", col_name] = "Unknown"
        chunk.loc[chunk[col_name] == "null", col_name] = "Unknown"
        chunk.loc[chunk[col_name] == "NULL", col_name] = "Unknown"
    
    processed_chunks.append(chunk)

# Combine all processed chunks into a single DataFrame
df = pd.concat(processed_chunks, ignore_index=True)



C:\Users\User\AppData\Local\Temp\ipykernel_15884\2670928087.py:29: DtypeWarning: Columns (8,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunk_iter:
C:\Users\User\AppData\Local\Temp\ipykernel_15884\2670928087.py:29: DtypeWarning: Columns (8,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunk_iter:
C:\Users\User\AppData\Local\Temp\ipykernel_15884\2670928087.py:29: DtypeWarning: Columns (8,18,20,31,32,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunk_iter:


In [3]:


# Convert date columns to datetime
df['Closed Date'] = pd.to_datetime(df['Closed Date'], errors='coerce')
df['Created Date'] = pd.to_datetime(df['Created Date'], errors='coerce')

# Calculate response time in seconds
df['response_time'] = (df['Closed Date'] - df['Created Date']).dt.total_seconds()

# Feature engineering: extract hour, day of week, month, year
df['hour'] = df['Created Date'].dt.hour
df['day_of_week'] = df['Created Date'].dt.dayofweek + 1  # Pandas: Monday=0, so add 1 if you need Sunday=1
df['month'] = df['Created Date'].dt.month
df['year'] = df['Created Date'].dt.year

# Define time_of_day
conditions = [
    (df['hour'] >= 6) & (df['hour'] < 12),
    (df['hour'] >= 12) & (df['hour'] < 18),
    (df['hour'] >= 18) & (df['hour'] < 22)
]
choices = ['morning', 'afternoon', 'evening']
df['time_of_day'] = np.select(conditions, choices, default='night')

# Define is_weekend (assuming day_of_week: 1=Monday, 7=Sunday)
df['is_weekend'] = np.where(df['day_of_week'].isin([6, 7]), 1, 0)


In [4]:

# Define numeric columns
numeric_cols = ["Incident Zip", "Latitude", "Longitude", "hour", "day_of_week", "month", "year", "is_weekend"]
# Convert suspected numeric columns to numeric type
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Optionally, handle any newly introduced NaN values (e.g., drop rows)
df = df.dropna(subset=numeric_cols + ['response_time'])

# Prepare the feature matrix (X) and label vector (y)
X = df[categorical_cols + numeric_cols]
y = df['response_time']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Remove 'lan' and 'lat' columns from the training set
X_train = X_train.drop(columns=['Longitude', 'Latitude'])
X_test = X_test.drop(columns=['Longitude', 'Latitude'])

# Use ColumnTransformer to one-hot encode categorical columns
ct = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'  # keep the numeric columns as is
)

# Fit and transform the training data and transform test data
X_train_enc = ct.fit_transform(X_train)
X_test_enc = ct.transform(X_test)

# Train a linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train_enc, y_train)

# Make predictions and evaluate RMSE
predictions = lr_model.predict(X_test_enc)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"Root Mean Squared Error (RMSE): {rmse}")


Root Mean Squared Error (RMSE): 37235437.97229638


In [5]:
# Compute the R² score (a measure of how well the model fits the data)
r2_score = lr_model.score(X_test_enc, y_test)
print(f"Model Accuracy (R² Score): {r2_score}")

Model Accuracy (R² Score): 0.5127225524511413
